# Uso de callbacks

En este ejercicio, implementaremos métodos de callbacks para detener el entrenamiento cuando se cumpla una métrica específica. Esta es una función útil, ya que permitirá no completar todas las épocas cuando se alcance este umbral. Por ejemplo, si configuramos 1000 épocas y la precisión deseada ya se alcanza en la época 200, el entrenamiento se detendrá automáticamente. Veamos cómo se implementa esto en las siguientes secciones.

## Cargamos y normalizamos el conjunto de datos MNIST de moda

Al igual que en el ejercicio anterior, volveremos a utilizar el conjunto de datos Fashion MNIST.Recuerda que debemos ormalizar los valores de píxeles para ayudar a optimizar el entrenamiento.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

# Instanciamos el conjunto de datos
mnist = tf.keras.datasets.fashion_mnist
# Cargamos el dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Normalizamos los valores


4422102/4422102 [==============================] - 0s 0us/step


In [2]:
train_images = train_images/255
test_images = test_images/255

## Creamos una clase callbacks

Puede crear un método callbacks definiendo una clase que herede la clase base [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback). Desde allí, pudemos definir los métodos disponibles para establecer dónde se ejecutará la evaluación del método. Por ejemplo, a continuación, utilizará el método [on_epoch_end()](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback#on_epoch_end) para verificar el valor de la loss function al final de la época.

In [3]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>=0.8):
      print("La precision ha alcanzado el 80%, se cancela el entrenamiento")
      self.model.stop_training = True

callbacks = myCallback()



## Definimos y compilamos el modelo

A continuación, definiremos y compilaremos el modelo. La arquitectura será similar a la que se construyó en el laboratorio anterior. Luego, estableceremos el optimizador, la loss function y las métricas que usaremos para el entrenamiento.

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics=['accuracy'])

### Entrenamos al modelo

Ahora estamos listos para entrenar el modelo. Para configurar la devolución de llamada, simplemente configure el parámetro `callbacks` en la instancia `myCallback` que declaramos anteriormente.

In [6]:
model.fit(train_images, train_labels, epochs=12, callbacks=[callbacks])

Epoch 1/12
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3447 - accuracy: 0.8748


Notarás que el entrenamiento no necesita completar las 10 épocas. Al evaluar el méotodo al final de la época, podemos verificar los parámetros de entrenamiento y comparar si cumplimos con el umbral establecido en la definición de la función. En este caso, simplemente se detendrá cuando la loss function caiga por debajo de `0.40` después de la época actual.

*Desafío opcional: modifica el código para que el entrenamiento se detenga cuando la métrica de precisión supere el 60 %.*